In [1]:
import cv2
import numpy as np
import time

from src.display import showarray

In [2]:
dice_mon = cv2.imread("data/images/rec_frame_c.jpg")

In [4]:
def get_dice_detector() -> cv2.SimpleBlobDetector:
    params = cv2.SimpleBlobDetector_Params()
    params.filterByArea = True
    params.minArea = 25
    params.maxArea = 60
    params.filterByCircularity = True
    params.minCircularity = 0.4
    params.filterByColor = True
    params.filterByConvexity = True
    params.minConvexity = 0.8
    params.filterByInertia = True
    params.minInertiaRatio = 0.7
    # params.minThreshold = 10
    # params.maxThreshold = 200
    # params.minInertiaRatio = 0.7
    return cv2.SimpleBlobDetector_create(params)

dice_detector = get_dice_detector()

In [5]:
def dilated_black_color_image(image: np.ndarray, threshold: int = 40) -> np.ndarray:
    gs_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gs_image[gs_image > threshold] = 255
    gs_image[gs_image <= threshold] = 0
    return gs_image

In [6]:
def detect_dice_dots(image: np.ndarray) -> list[cv2.KeyPoint]:
    return dice_detector.detect(dilated_black_color_image(image))

def calculate_frame_similarity(frame_1: np.ndarray, frame_2: np.ndarray) -> float:
    if frame_1 is None:
        return 1
    last_frame_gs = cv2.cvtColor(frame_1, cv2.COLOR_BGR2GRAY) / 255
    current_frame_gs = cv2.cvtColor(frame_2, cv2.COLOR_BGR2GRAY) / 255
    last_frame_norm = last_frame_gs / np.sqrt(np.sum(last_frame_gs ** 2))
    current_frame_norm = current_frame_gs / np.sqrt(np.sum(current_frame_gs ** 2))
    return np.sum(last_frame_norm * current_frame_norm)

In [8]:
capture = cv2.VideoCapture("data/recordings/Easy_1.mp4")
cv2.namedWindow("Feed", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Feed", 400, 800)
last_frame = None
sim_times = []
detect_times = []
while(capture.isOpened()):
    ret, frame = capture.read()
    if not ret:
        break
    similarity = calculate_frame_similarity(last_frame, frame)
    last_frame = frame
    keypoints = detect_dice_dots(frame)
    frame = cv2.drawKeypoints(frame, keypoints, np.array([]), (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    frame_resized = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    cv2.imshow('Feed', frame) 
    if cv2.waitKey(3) & 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()